In [2]:
import pandas as pd
import os
os.getcwd()
os.chdir('/mnt/dolphinfs/hdd_pool/docker/user/hadoop-aipnlp/liujincheng06/task4')
#from call import generate_answer_useGpt4

In [3]:
from call import generate_answer_useGpt4

In [4]:
df = pd.read_excel("mbpp_js.xlsx")

In [5]:
codes = df['code_str']

In [6]:
assert_stmts = df['test']

In [7]:
codes_with_test_assert = []
for i in range(len(codes)):
    codes_with_test_assert.append(codes[i] + "\n" + assert_stmts[i])

In [8]:
print(codes_with_test_assert[0])

def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

assert remove_Occ("hellolloll","l") == "helollol"
assert remove_Occ("","l") == ""



In [9]:
transferd_codes = [df['js_code_str'][i] for i in range(len(df))]
transferd_tests = [df['js_test'][i] for i in range(len(df))]
back = [df['backup'][i] for i in range(len(df))]

In [10]:
import re

def extract_js_sections(code):
    # 匹配 main 函数部分
    main_pattern = re.compile(r'const test.+', re.DOTALL)
    main_function = main_pattern.search(code)
    main_function = main_function.group() if main_function else ""

    code_str = main_pattern.sub("",code)

    return code_str,main_function

In [11]:
from tqdm import tqdm

In [12]:
import json

with open("mbpp_cpp_tests.jsonl","w") as f:
    for i,v in tqdm(enumerate(codes_with_test_assert)):
        if len(str(back[i])) < 5:
            prompt = """你的任务是进行代码翻译，源代码语言为python，翻译后代码语言为javascript。
        代码中含有一个函数，一些测试用例，可以直接运行。
        要求翻译后的代码也能直接运行。
        待翻译的python代码内容:
        %s

        要求:
        1. 只需要给出翻译后的代码，不需要文字解释；
        2. 请按照指定格式给出代码
        3. 直接给出代码，不要markdown脚本语言
        4. 注意导入包的完整性
        5. 输出的时候去掉我给你的提示信息，只输出代码

        格式:
        //function difinition

        
        const test{function_name} = () => {
            //test cases
            //use
        }
        test{function_name}() // invoke test
        //function_name is the function name you difined before(be same as the function name of python code)
                                
                                """ %v
            generation = generate_answer_useGpt4(prompt)
            code_str,test = extract_js_sections(generation)
            f.write(json.dumps({'cpp_code_str':code_str,'cpp_test':test,'backup':generation}) + "\n")
            transferd_codes[i] = code_str
            transferd_tests[i] = test
            back[i] = generation
        

        

500it [03:20,  2.50it/s]


In [13]:
df['js_code_str'] = transferd_codes
df['js_test'] = transferd_tests
df['backup'] = back

In [14]:
df.to_excel('mbpp_js.xlsx')